In [ ]:
# 纤维截面定义
def Wsection(secID, matID, d, bf, tf, tw, nfdw, nftw, nfbf, nftf):
    """
    create a standard W section given the nominal section properties
    :param secID: 截面编号
    :param matID: 材料编号
    :param d: 截面高度
    :param bf: 腹板厚度
    :param tf: 翼缘宽度
    :param tw: 翼缘厚度
    :param nfdw: 沿腹板高度的纤维数量
    :param nftw: 沿腹板厚度的纤维数量
    :param nfbf: 沿翼缘宽度的纤维数量
    :param nftf: 沿翼缘厚度的纤维数量
    :return:
    """
    dw = d - 2 * tf
    y1 = -d / 2
    y2 = -dw / 2
    y3 = dw / 2
    y4 = d / 2

    z1 = -bf / 2
    z2 = -tw / 2
    z3 = tw / 2
    z4 = bf / 2

    section('Fiber', secID)
    patch('quad', matID, nfbf, nftf, y1, z4, y1, z1, y2, z1, y2, z4)
    patch('quad', matID, nftw, nfdw, y2, z3, y2, z2, y3, z2, y3, z3)
    patch('quad', matID, nfbf, nftf, y3, z4, y3, z1, y4, z1, y4, z4)

In [ ]:
# 定义惯容系统
def inerterDamper2DX(inode, jnode, cd, md, ks):
    """
    定义TVMD
    :param inode: 连接节点i
    :param jnode: 连接节点j
    :param cd: 阻尼系数
    :param md: 惯容系数
    :param ks: 刚度
    """

    # set nodes
    x1 = nodeCoord(inode, 1)
    y1 = nodeCoord(inode, 2)
    x2 = nodeCoord(jnode, 1)
    y2 = nodeCoord(jnode, 2)
    rigid_beam_len = 100.0  # 刚性杆的长度

    node_start = inode * 100 + jnode * 10
    node(node_start + 1, x1, y1)
    node(node_start + 2, x1, y1)
    node(node_start + 3, x2, y2)
    node(node_start + 4, x1, y1 - rigid_beam_len)
    node(node_start + 5, x1 - 1, y1 - rigid_beam_len)

    fix(node_start + 1, 0, 1, 1)
    fix(node_start + 2, 0, 1, 0)
    fix(node_start + 3, 0, 1, 1)
    fix(node_start + 5, 0, 1, 0)

    equalDOF(inode, node_start + 1, 1)
    equalDOF(inode, node_start + 4, 1)
    equalDOF(jnode, node_start + 3, 1)

    # dashpot
    mat_start = inode * 100 + jnode * 10
    ele_start = inode * 100 + jnode * 10
    uniaxialMaterial('Viscous', mat_start + 1, cd, 1)
    element('zeroLength', ele_start + 1, node_start + 1, node_start + 2, '-mat', mat_start + 1, '-dir', 1)  # 该构件的方向是x向

    # mass
    mass(node_start + 4, 1e-5, 1e-5, rigid_beam_len * rigid_beam_len * md)

    # Spring
    if x1 == x2 and y1 == y2:
        uniaxialMaterial('Elastic', mat_start + 2, ks)
        element('zeroLength', ele_start + 2, node_start + 2, node_start + 3, '-mat', mat_start + 2, '-dir',
                1)  # 弹簧方向是x向
    else:
        spring_len = np.sqrt((x1 - x2) * (x1 - x2) + (y1 - y2) * (y1 - y2))
        uniaxialMaterial('Elastic', mat_start + 2, ks * spring_len)
        element('corotTruss', ele_start + 2, node_start + 2, node_start + 3, 1.0, mat_start + 2)

    # geomTransf Linear 1;
    # Rigid beam
    element('elasticBeamColumn', ele_start + 3, node_start + 2, node_start + 4, 1.0e+10, 1.0, 1.0e+12, 1)
    element('elasticBeamColumn', ele_start + 4, node_start + 5, node_start + 4, 1.0e+12, 1.0, 1.0e-8, 1)

In [ ]:
def drift_cal(i_node, height):
    """
    计算层间位移角
    :param i_node: i节点的节点标志
    :param height: 层高
    :return:
    """
    # j_node是i_node的下层节点，tag比i_node小10
    j_node = i_node - 10
    drift = (nodeDisp(i_node, 1) - nodeDisp(j_node, 1)) / height
    return drift


In [ ]:
def height(node_tag):
    """
    返回节点标志对应的层高
    :param node_tag: 节点标志
    :return: 层高
    """
    if node_tag < 20:
        return 5.4864  # 1层
    else:
        return 3.9624  # 2~9层

In [ ]:
def cal_drift(drift):
    """
    计算层间位移角指标
    :param drift: 结构各层的层间位移角时程
    :return: 各层中最大的最大位移角和平均的最大位移角
    """
    drift_each_max = np.max(drift, 1)
    return drift_each_max

In [ ]:
def original_model(wave_file, NPTS, dt, PGA):
    """
    构建非线性模型，计算目标参数
    :param wave_file: 地震波文件名
    :param NPTS: 地震波的记录节点数量
    :param dt: 地震波的时间间隔
    :param PGA: 地震波的放大系数
    :return: [T_1, a_max, a_avg, drift_max, drift_avg]
    """
    wipe()
    model('basic', '-ndm', 2, '-ndf', 3)

    # 框架节点
    h1 = 5.4864  # 1层层高
    h2 = 3.9624  # 2-9层层高
    span_len = 9.144  # 跨长

    for i in range(0, 10):
        for j in range(1, 7):
            if i == 0 or i == 1:
                h = i * h1
            else:
                h = h1 + (i - 1) * h2
            node_tag = 10 * i + j
            x = span_len * (j - 1)
            y = h
            node(node_tag, x, y)

    # 节点约束
    for i in range(0, 10):
        for j in range(1, 7):
            if i == 0:
                fix(i * 10 + j, 1, 1, 1)
            else:
                fix(i * 10 + j, 0, 0, 0)

    # 变截面节点
    bh1 = 1.8288 + h1
    bh2 = 1.8288 + h1 + 2 * h2
    bh3 = 1.8288 + h1 + 4 * h2
    bh4 = 1.8288 + h1 + 6 * h2

    for i in range(1, 7):
        node_tag = 100 + i
        x = span_len * (i - 1)
        y = bh1
        node(node_tag, x, y)
        fix(node_tag, 0, 0, 0)

    for i in range(1, 7):
        node_tag = 200 + i
        x = span_len * (i - 1)
        y = bh2
        node(node_tag, x, y)
        fix(node_tag, 0, 0, 0)

    for i in range(1, 7):
        node_tag = 300 + i
        x = span_len * (i - 1)
        y = bh3
        node(node_tag, x, y)
        fix(node_tag, 0, 0, 0)

    for i in range(1, 7):
        node_tag = 400 + i
        x = span_len * (i - 1)
        y = bh4
        node(node_tag, x, y)
        fix(node_tag, 0, 0, 0)


    # 材料定义
    uniaxialMaterial('Steel01', 1, 345000000, 200000000000, 0.01)  # 柱材料
    uniaxialMaterial('Steel01', 2, 248000000, 200000000000, 0.01)  # 材料
    uniaxialMaterial('Elastic', 3, 1.0e20)


    # 纤维截面定义
    # 柱
    # W14*257
    sectionID = 11
    materialID = 1
    d = 0.4161
    bf = 0.4063
    tf = 0.048
    tw = 0.0298
    nfdw = 20
    nftw = 2
    nfbf = 20
    nftf = 2
    Wsection(sectionID, materialID, d, bf, tf, tw, nfdw, nftw, nfbf, nftf)

    # W14*283
    sectionID = 12
    materialID = 1
    d = 0.4252
    bf = 0.4092
    tf = 0.0526
    tw = 0.0328
    nfdw = 20
    nftw = 2
    nfbf = 20
    nftf = 2
    Wsection(sectionID, materialID, d, bf, tf, tw, nfdw, nftw, nfbf, nftf)
    # W14*370
    sectionID = 13
    materialID = 1
    d = 0.4552
    bf = 0.4185
    tf = 0.0676
    tw = 0.042
    nfdw = 20
    nftw = 2
    nfbf = 20
    nftf = 2
    Wsection(sectionID, materialID, d, bf, tf, tw, nfdw, nftw, nfbf, nftf)
    # W14*455
    sectionID = 14
    materialID = 1
    d = 0.4831
    bf = 0.4276
    tf = 0.0815
    tw = 0.0512
    nfdw = 20
    nftw = 2
    nfbf = 20
    nftf = 2
    Wsection(sectionID, materialID, d, bf, tf, tw, nfdw, nftw, nfbf, nftf)
    # W14*500
    sectionID = 15
    materialID = 1
    d = 0.4978
    bf = 0.4321
    tf = 0.0889
    tw = 0.0556
    nfdw = 20
    nftw = 2
    nfbf = 20
    nftf = 2
    Wsection(sectionID, materialID, d, bf, tf, tw, nfdw, nftw, nfbf, nftf)
    # 梁
    # W24*68
    sectionID = 21
    materialID = 2
    d = 0.6027
    bf = 0.2277
    tf = 0.0149
    tw = 0.0105
    nfdw = 20
    nftw = 2
    nfbf = 20
    nftf = 2
    Wsection(sectionID, materialID, d, bf, tf, tw, nfdw, nftw, nfbf, nftf)
    # W27*84
    sectionID = 22
    materialID = 2
    d = 0.6784
    bf = 0.253
    tf = 0.0163
    tw = 0.0117
    nfdw = 20
    nftw = 2
    nfbf = 20
    nftf = 2
    Wsection(sectionID, materialID, d, bf, tf, tw, nfdw, nftw, nfbf, nftf)

    # W30*99
    sectionID = 23
    materialID = 2
    d = 0.7531
    bf = 0.2654
    tf = 0.017
    tw = 0.0132
    nfdw = 20
    nftw = 2
    nfbf = 20
    nftf = 2
    Wsection(sectionID, materialID, d, bf, tf, tw, nfdw, nftw, nfbf, nftf)
    # W36*135
    sectionID = 24
    materialID = 2
    d = 0.903
    bf = 0.3035
    tf = 0.0201
    tw = 0.0152
    nfdw = 20
    nftw = 2
    nfbf = 20
    nftf = 2
    Wsection(sectionID, materialID, d, bf, tf, tw, nfdw, nftw, nfbf, nftf)
    # W36*160
    sectionID = 25
    materialID = 2
    d = 0.9147
    bf = 0.3048
    tf = 0.0259
    tw = 0.0165
    nfdw = 20
    nftw = 2
    nfbf = 20
    nftf = 2
    Wsection(sectionID, materialID, d, bf, tf, tw, nfdw, nftw, nfbf, nftf)

    # 建立截面
    section('Aggregator', 111, 3, 'T', '-section', 11)
    section('Aggregator', 112, 3, 'T', '-section', 12)
    section('Aggregator', 113, 3, 'T', '-section', 13)
    section('Aggregator', 114, 3, 'T', '-section', 14)
    section('Aggregator', 115, 3, 'T', '-section', 15)
    section('Aggregator', 121, 3, 'T', '-section', 21)
    section('Aggregator', 122, 3, 'T', '-section', 22)
    section('Aggregator', 123, 3, 'T', '-section', 23)
    section('Aggregator', 124, 3, 'T', '-section', 24)
    section('Aggregator', 125, 3, 'T', '-section', 25)

    # 定义构件
    geomTransf('Linear', 1)
    geomTransf('Linear', 2)

    # 柱
    # 1
    for i in range(1, 7):
        element('nonlinearBeamColumn', 10 + i, i, 10 + i, 5, 115, 1)

    # 1.5
    for i in range(1, 7):
        element('nonlinearBeamColumn', 20 + i, 10 + i, 100 + i, 5, 115, 1)

    # 2
    for i in range(1, 7):
        element('nonlinearBeamColumn', 30 + i, 100 + i, 20 + i, 5, 114, 1)

    # 3
    for i in range(1, 7):
        element('nonlinearBeamColumn', 40 + i, 20 + i, 30 + i, 5, 114, 1)

    # 3.5
    for i in range(1, 7):
        element('nonlinearBeamColumn', 50 + i, 30 + i, 200 + i, 5, 114, 1)

    # 4
    for i in range(1, 7):
        element('nonlinearBeamColumn', 60 + i, 200 + i, 40 + i, 5, 113, 1)

    # 5
    for i in range(1, 7):
        element('nonlinearBeamColumn', 70 + i, 40 + i, 50 + i, 5, 113, 1)

    # 5.5
    for i in range(1, 7):
        element('nonlinearBeamColumn', 80 + i, 50 + i, 300 + i, 5, 113, 1)

    # 6
    for i in range(1, 7):
        element('nonlinearBeamColumn', 90 + i, 300 + i, 60 + i, 5, 112, 1)

    # 7
    for i in range(1, 7):
        element('nonlinearBeamColumn', 100 + i, 60 + i, 70 + i, 5, 112, 1)

    # 7.5
    for i in range(1, 7):
        element('nonlinearBeamColumn', 110 + i, 70 + i, 400 + i, 5, 112, 1)

    # 8
    for i in range(1, 7):
        element('nonlinearBeamColumn', 120 + i, 400 + i, 80 + i, 5, 111, 1)

    # 9
    for i in range(1, 7):
        element('nonlinearBeamColumn', 130 + i, 80 + i, 90 + i, 5, 111, 1)


    # 梁
    # 1
    for i in range(1, 6):
        element('nonlinearBeamColumn', 210 + i, 10 + i, 10 + i + 1, 5, 125, 2)

    # 2
    for i in range(1, 6):
        element('nonlinearBeamColumn', 220 + i, 20 + i, 20 + i + 1, 5, 125, 2)

    # 3
    for i in range(1, 6):
        element('nonlinearBeamColumn', 230 + i, 30 + i, 30 + i + 1, 5, 124, 2)

    # 4
    for i in range(1, 6):
        element('nonlinearBeamColumn', 240 + i, 40 + i, 40 + i + 1, 5, 124, 2)

    # 5
    for i in range(1, 6):
        element('nonlinearBeamColumn', 250 + i, 50 + i, 50 + i + 1, 5, 124, 2)

    # 6
    for i in range(1, 6):
        element('nonlinearBeamColumn', 260 + i, 60 + i, 60 + i + 1, 5, 124, 2)

    # 7
    for i in range(1, 6):
        element('nonlinearBeamColumn', 270 + i, 70 + i, 70 + i + 1, 5, 123, 2)

    # 8
    for i in range(1, 6):
        element('nonlinearBeamColumn', 280 + i, 80 + i, 80 + i + 1, 5, 122, 2)

    # 9
    for i in range(1, 6):
        element('nonlinearBeamColumn', 290 + i, 90 + i, 90 + i + 1, 5, 121, 2)


    # 定义节点质量
    # 1层
    mass1 = 50465.75
    mass2 = 100931.5
    for i in range(1, 7):
        if i == 1 or i == 6:
            m = mass1
        else:
            m = mass2
        node_tag = 10 + i
        mass(node_tag, m, m, m)


    # 2-8层
    mass1 = 49416.46
    mass2 = 98832.93
    for j in range(2, 9):
        for i in range(1, 7):
            if i == 1 or i == 6:
                m = mass1
            else:
                m = mass2
            node_tag = j * 10 + i
            mass(node_tag, m, m, m)

    # 9层
    mass1 = 53463.72
    mass2 = 106927.4
    for i in range(1, 7):
        if i == 1 or i == 6:
            m = mass1
        else:
            m = mass2
        node_tag = 90 + i
        mass(node_tag, m, m, m)


    wipeAnalysis()
    # 特征值分析
    num_eigen = 10
    eigen_values = eigen(num_eigen)
    w2 = eigen_values[0]
    T_1 = 2 * np.pi / np.sqrt(w2)

    # 定义瑞利阻尼
    xDamp = 0.04  # 钢结构的阻尼比
    MpropSwitch = 1.0
    KcurrSwitch = 0.0
    KcommSwitch = 1.0
    KinitSwitch = 0.0
    omega1 = np.sqrt(eigen_values[0])
    omega3 = np.sqrt(eigen_values[2])
    alphaM = MpropSwitch * xDamp * (2 * omega1 * omega3) / (omega1 + omega3)
    betaKcurr = KcurrSwitch * 2. * xDamp / (omega1 + omega3)
    betaKcomm = KcommSwitch * 2. * xDamp / (omega1 + omega3)
    betaKinit = KinitSwitch * 2. * xDamp / (omega1 + omega3)
    rayleigh(alphaM, betaKcurr, betaKinit, betaKcomm)

    # 动力学分析
    load_tag_dynamic = 1
    pattern_tag_dynamic = 1
    X = 1

    timeSeries('Path', load_tag_dynamic, '-filePath', wave_file, '-dt', dt, '-factor', PGA)
    pattern('UniformExcitation', pattern_tag_dynamic, X, '-accel', load_tag_dynamic)

    analysis_dt = 0.01
    final_time = (NPTS - 1) * dt

    # NEWMARK直接积分
    wipeAnalysis()
    constraints('Plain')  # 约束条件
    numberer('RCM')  # 对节点重新编号
    system('BandGeneral')  # 解矩阵的方法
    test('EnergyIncr', 1.000000e-6, 10, 0, 2)  # 误差分析
    algorithm('KrylovNewton')  # 算法
    integrator('Newmark', 0.5, 0.25)  # 用newmark法积分
    analysis('Transient')  # 时程分析

    # 取时程数据的节点
    mark_nodes = [11, 21, 31, 41, 51, 61, 71, 81, 91]
    mark_nodes_len = len(mark_nodes)
    time_num = int(final_time / analysis_dt) + 100
    time = np.zeros(time_num)
    acceleration = np.zeros([mark_nodes_len, time_num])
    drift = np.zeros([mark_nodes_len, time_num])
    ok = 0
    time_idx = 0

    while ok == 0 and getTime() < final_time:
        current_time = getTime()
        ok = analyze(1, analysis_dt)
        for layer_idx in range(mark_nodes_len):
            acceleration[layer_idx, time_idx] = nodeAccel(mark_nodes[layer_idx], 1)
            drift[layer_idx, time_idx] = drift_cal(mark_nodes[layer_idx], height(mark_nodes[layer_idx]))
        time[time_idx] = current_time
        time_idx += 1
    drift_each = cal_drift(drift)
    return [acceleration, drift_each]

In [ ]:
def TVMD_model(md, kd, cd, wave_file, NPTS, dt, PGA):
    """
    构建非线性模型，计算目标参数
    :param md: 每层TVMD的惯容系数
    :param kd: 每层TVMD的刚度
    :param cd: 每层TVMD的阻尼系数
    :param wave_file: 地震波文件名
    :param NPTS: 地震波的记录节点数量
    :param dt: 地震波的时间间隔
    :param PGA: 地震波的放大系数
    :return: [T_1, a_max, a_avg, drift_max, drift_avg]
    """
    wipe()
    model('basic', '-ndm', 2, '-ndf', 3)

    # 框架节点
    h1 = 5.4864  # 1层层高
    h2 = 3.9624  # 2-9层层高
    span_len = 9.144  # 跨长

    for i in range(0, 10):
        for j in range(1, 7):
            if i == 0 or i == 1:
                h = i * h1
            else:
                h = h1 + (i - 1) * h2
            node_tag = 10 * i + j
            x = span_len * (j - 1)
            y = h
            node(node_tag, x, y)

    # 节点约束
    for i in range(0, 10):
        for j in range(1, 7):
            if i == 0:
                fix(i * 10 + j, 1, 1, 1)
            else:
                fix(i * 10 + j, 0, 0, 0)

    # 变截面节点
    bh1 = 1.8288 + h1
    bh2 = 1.8288 + h1 + 2 * h2
    bh3 = 1.8288 + h1 + 4 * h2
    bh4 = 1.8288 + h1 + 6 * h2

    for i in range(1, 7):
        node_tag = 100 + i
        x = span_len * (i - 1)
        y = bh1
        node(node_tag, x, y)
        fix(node_tag, 0, 0, 0)

    for i in range(1, 7):
        node_tag = 200 + i
        x = span_len * (i - 1)
        y = bh2
        node(node_tag, x, y)
        fix(node_tag, 0, 0, 0)

    for i in range(1, 7):
        node_tag = 300 + i
        x = span_len * (i - 1)
        y = bh3
        node(node_tag, x, y)
        fix(node_tag, 0, 0, 0)

    for i in range(1, 7):
        node_tag = 400 + i
        x = span_len * (i - 1)
        y = bh4
        node(node_tag, x, y)
        fix(node_tag, 0, 0, 0)


    # 材料定义
    uniaxialMaterial('Steel01', 1, 345000000, 200000000000, 0.01)  # 柱材料
    uniaxialMaterial('Steel01', 2, 248000000, 200000000000, 0.01)  # 材料
    uniaxialMaterial('Elastic', 3, 1.0e20)


    # 纤维截面定义
    # 柱
    # W14*257
    sectionID = 11
    materialID = 1
    d = 0.4161
    bf = 0.4063
    tf = 0.048
    tw = 0.0298
    nfdw = 20
    nftw = 2
    nfbf = 20
    nftf = 2
    Wsection(sectionID, materialID, d, bf, tf, tw, nfdw, nftw, nfbf, nftf)

    # W14*283
    sectionID = 12
    materialID = 1
    d = 0.4252
    bf = 0.4092
    tf = 0.0526
    tw = 0.0328
    nfdw = 20
    nftw = 2
    nfbf = 20
    nftf = 2
    Wsection(sectionID, materialID, d, bf, tf, tw, nfdw, nftw, nfbf, nftf)
    # W14*370
    sectionID = 13
    materialID = 1
    d = 0.4552
    bf = 0.4185
    tf = 0.0676
    tw = 0.042
    nfdw = 20
    nftw = 2
    nfbf = 20
    nftf = 2
    Wsection(sectionID, materialID, d, bf, tf, tw, nfdw, nftw, nfbf, nftf)
    # W14*455
    sectionID = 14
    materialID = 1
    d = 0.4831
    bf = 0.4276
    tf = 0.0815
    tw = 0.0512
    nfdw = 20
    nftw = 2
    nfbf = 20
    nftf = 2
    Wsection(sectionID, materialID, d, bf, tf, tw, nfdw, nftw, nfbf, nftf)
    # W14*500
    sectionID = 15
    materialID = 1
    d = 0.4978
    bf = 0.4321
    tf = 0.0889
    tw = 0.0556
    nfdw = 20
    nftw = 2
    nfbf = 20
    nftf = 2
    Wsection(sectionID, materialID, d, bf, tf, tw, nfdw, nftw, nfbf, nftf)
    # 梁
    # W24*68
    sectionID = 21
    materialID = 2
    d = 0.6027
    bf = 0.2277
    tf = 0.0149
    tw = 0.0105
    nfdw = 20
    nftw = 2
    nfbf = 20
    nftf = 2
    Wsection(sectionID, materialID, d, bf, tf, tw, nfdw, nftw, nfbf, nftf)
    # W27*84
    sectionID = 22
    materialID = 2
    d = 0.6784
    bf = 0.253
    tf = 0.0163
    tw = 0.0117
    nfdw = 20
    nftw = 2
    nfbf = 20
    nftf = 2
    Wsection(sectionID, materialID, d, bf, tf, tw, nfdw, nftw, nfbf, nftf)

    # W30*99
    sectionID = 23
    materialID = 2
    d = 0.7531
    bf = 0.2654
    tf = 0.017
    tw = 0.0132
    nfdw = 20
    nftw = 2
    nfbf = 20
    nftf = 2
    Wsection(sectionID, materialID, d, bf, tf, tw, nfdw, nftw, nfbf, nftf)
    # W36*135
    sectionID = 24
    materialID = 2
    d = 0.903
    bf = 0.3035
    tf = 0.0201
    tw = 0.0152
    nfdw = 20
    nftw = 2
    nfbf = 20
    nftf = 2
    Wsection(sectionID, materialID, d, bf, tf, tw, nfdw, nftw, nfbf, nftf)
    # W36*160
    sectionID = 25
    materialID = 2
    d = 0.9147
    bf = 0.3048
    tf = 0.0259
    tw = 0.0165
    nfdw = 20
    nftw = 2
    nfbf = 20
    nftf = 2
    Wsection(sectionID, materialID, d, bf, tf, tw, nfdw, nftw, nfbf, nftf)

    # 建立截面
    section('Aggregator', 111, 3, 'T', '-section', 11)
    section('Aggregator', 112, 3, 'T', '-section', 12)
    section('Aggregator', 113, 3, 'T', '-section', 13)
    section('Aggregator', 114, 3, 'T', '-section', 14)
    section('Aggregator', 115, 3, 'T', '-section', 15)
    section('Aggregator', 121, 3, 'T', '-section', 21)
    section('Aggregator', 122, 3, 'T', '-section', 22)
    section('Aggregator', 123, 3, 'T', '-section', 23)
    section('Aggregator', 124, 3, 'T', '-section', 24)
    section('Aggregator', 125, 3, 'T', '-section', 25)

    # 定义构件
    geomTransf('Linear', 1)
    geomTransf('Linear', 2)

    # 柱
    # 1
    for i in range(1, 7):
        element('nonlinearBeamColumn', 10 + i, i, 10 + i, 5, 115, 1)

    # 1.5
    for i in range(1, 7):
        element('nonlinearBeamColumn', 20 + i, 10 + i, 100 + i, 5, 115, 1)

    # 2
    for i in range(1, 7):
        element('nonlinearBeamColumn', 30 + i, 100 + i, 20 + i, 5, 114, 1)

    # 3
    for i in range(1, 7):
        element('nonlinearBeamColumn', 40 + i, 20 + i, 30 + i, 5, 114, 1)

    # 3.5
    for i in range(1, 7):
        element('nonlinearBeamColumn', 50 + i, 30 + i, 200 + i, 5, 114, 1)

    # 4
    for i in range(1, 7):
        element('nonlinearBeamColumn', 60 + i, 200 + i, 40 + i, 5, 113, 1)

    # 5
    for i in range(1, 7):
        element('nonlinearBeamColumn', 70 + i, 40 + i, 50 + i, 5, 113, 1)

    # 5.5
    for i in range(1, 7):
        element('nonlinearBeamColumn', 80 + i, 50 + i, 300 + i, 5, 113, 1)

    # 6
    for i in range(1, 7):
        element('nonlinearBeamColumn', 90 + i, 300 + i, 60 + i, 5, 112, 1)

    # 7
    for i in range(1, 7):
        element('nonlinearBeamColumn', 100 + i, 60 + i, 70 + i, 5, 112, 1)

    # 7.5
    for i in range(1, 7):
        element('nonlinearBeamColumn', 110 + i, 70 + i, 400 + i, 5, 112, 1)

    # 8
    for i in range(1, 7):
        element('nonlinearBeamColumn', 120 + i, 400 + i, 80 + i, 5, 111, 1)

    # 9
    for i in range(1, 7):
        element('nonlinearBeamColumn', 130 + i, 80 + i, 90 + i, 5, 111, 1)


    # 梁
    # 1
    for i in range(1, 6):
        element('nonlinearBeamColumn', 210 + i, 10 + i, 10 + i + 1, 5, 125, 2)

    # 2
    for i in range(1, 6):
        element('nonlinearBeamColumn', 220 + i, 20 + i, 20 + i + 1, 5, 125, 2)

    # 3
    for i in range(1, 6):
        element('nonlinearBeamColumn', 230 + i, 30 + i, 30 + i + 1, 5, 124, 2)

    # 4
    for i in range(1, 6):
        element('nonlinearBeamColumn', 240 + i, 40 + i, 40 + i + 1, 5, 124, 2)

    # 5
    for i in range(1, 6):
        element('nonlinearBeamColumn', 250 + i, 50 + i, 50 + i + 1, 5, 124, 2)

    # 6
    for i in range(1, 6):
        element('nonlinearBeamColumn', 260 + i, 60 + i, 60 + i + 1, 5, 124, 2)

    # 7
    for i in range(1, 6):
        element('nonlinearBeamColumn', 270 + i, 70 + i, 70 + i + 1, 5, 123, 2)

    # 8
    for i in range(1, 6):
        element('nonlinearBeamColumn', 280 + i, 80 + i, 80 + i + 1, 5, 122, 2)

    # 9
    for i in range(1, 6):
        element('nonlinearBeamColumn', 290 + i, 90 + i, 90 + i + 1, 5, 121, 2)


    # 定义节点质量
    # 1层
    mass1 = 50465.75
    mass2 = 100931.5
    for i in range(1, 7):
        if i == 1 or i == 6:
            m = mass1
        else:
            m = mass2
        node_tag = 10 + i
        mass(node_tag, m, m, m)


    # 2-8层
    mass1 = 49416.46
    mass2 = 98832.93
    for j in range(2, 9):
        for i in range(1, 7):
            if i == 1 or i == 6:
                m = mass1
            else:
                m = mass2
            node_tag = j * 10 + i
            mass(node_tag, m, m, m)

    # 9层
    mass1 = 53463.72
    mass2 = 106927.4
    for i in range(1, 7):
        if i == 1 or i == 6:
            m = mass1
        else:
            m = mass2
        node_tag = 90 + i
        mass(node_tag, m, m, m)


    # 惯容系统参数*
    # 1
    inerterDamper2DX(3, 14, cd[0], md[0], kd[0])
    # 2
    inerterDamper2DX(13, 24, cd[1], md[1], kd[1])
    # 3
    inerterDamper2DX(23, 34, cd[2], md[2], kd[2])
    # 4
    inerterDamper2DX(33, 44, cd[3], md[3], kd[3])
    # 5
    inerterDamper2DX(43, 54, cd[4], md[4], kd[4])
    # 6
    inerterDamper2DX(53, 64, cd[5], md[5], kd[5])
    # 7
    inerterDamper2DX(63, 74, cd[6], md[6], kd[6])
    # 8
    inerterDamper2DX(73, 84, cd[7], md[7], kd[7])
    # 9
    inerterDamper2DX(83, 94, cd[8], md[8], kd[8])

    wipeAnalysis()
    # 特征值分析
    num_eigen = 10
    eigen_values = eigen(num_eigen)
    w2 = eigen_values[0]
    T_1 = 2 * np.pi / np.sqrt(w2)

    # 定义瑞利阻尼
    xDamp = 0.04  # 钢结构的阻尼比
    MpropSwitch = 1.0
    KcurrSwitch = 0.0
    KcommSwitch = 1.0
    KinitSwitch = 0.0
    omega1 = np.sqrt(eigen_values[0])
    omega3 = np.sqrt(eigen_values[2])
    alphaM = MpropSwitch * xDamp * (2 * omega1 * omega3) / (omega1 + omega3)
    betaKcurr = KcurrSwitch * 2. * xDamp / (omega1 + omega3)
    betaKcomm = KcommSwitch * 2. * xDamp / (omega1 + omega3)
    betaKinit = KinitSwitch * 2. * xDamp / (omega1 + omega3)
    rayleigh(alphaM, betaKcurr, betaKinit, betaKcomm)

    # 动力学分析
    load_tag_dynamic = 1
    pattern_tag_dynamic = 1
    X = 1

    timeSeries('Path', load_tag_dynamic, '-filePath', wave_file, '-dt', dt, '-factor', PGA)
    pattern('UniformExcitation', pattern_tag_dynamic, X, '-accel', load_tag_dynamic)

    analysis_dt = 0.01
    final_time = (NPTS - 1) * dt

    # NEWMARK直接积分
    wipeAnalysis()
    constraints('Plain')  # 约束条件
    numberer('RCM')  # 对节点重新编号
    system('BandGeneral')  # 解矩阵的方法
    test('EnergyIncr', 1.000000e-6, 10, 0, 2)  # 误差分析
    algorithm('KrylovNewton')  # 算法
    integrator('Newmark', 0.5, 0.25)  # 用newmark法积分
    analysis('Transient')  # 时程分析

    # 取时程数据的节点
    mark_nodes = [11, 21, 31, 41, 51, 61, 71, 81, 91]
    mark_nodes_len = len(mark_nodes)
    time_num = int(final_time / analysis_dt) + 100
    time = np.zeros(time_num)
    acceleration = np.zeros([mark_nodes_len, time_num])
    drift = np.zeros([mark_nodes_len, time_num])
    ok = 0
    time_idx = 0

    while ok == 0 and getTime() < final_time:
        current_time = getTime()
        ok = analyze(1, analysis_dt)
        for layer_idx in range(mark_nodes_len):
            acceleration[layer_idx, time_idx] = nodeAccel(mark_nodes[layer_idx], 1)
            drift[layer_idx, time_idx] = drift_cal(mark_nodes[layer_idx], height(mark_nodes[layer_idx]))
        time[time_idx] = current_time
        time_idx += 1

    drift_each = cal_drift(drift)
    return [acceleration, drift_each]

In [ ]:
wave_name = "Artificial_EQSignal2-Acc"
dt = 0.02
NPTS = 2000
wave_dir = "./wave_normalize/"
wave_file = wave_dir + wave_name + '.txt'
a, drift = original_model(wave_file, NPTS, dt, 5.1)

In [ ]:
analyze_dt = 0.01
time_nodes = a.shape[1]

In [ ]:
time = np.arange(start=0, stop=analyze_dt * time_nodes, step=analyze_dt, dtype=float)

In [ ]:
plt.plot(time, a[0])

In [ ]:
layer = np.arange(start=1, stop=10, step=1, dtype=int)

In [ ]:
plt.plot(drift, layer)

In [ ]:
a_c, drift_c = TVMD_model([0.94880083,0.63691009,0.87888782,0.45397529,0.2351799,0.9602562,0.33591441,0.95306617,0.72680965], [0.16141806,0.74959667,0.06038833,0.70491292,0.01887849,0.84438961,0.57389385,0.88525194,0.73292641], [0.16998586,0.41742601,0.29893473,0.5473103,0.48232276,0.81385777,0.94395333,0.75298743,0.58476329], wave_file, NPTS, dt, 5.1)

In [ ]:
# 绘图颜色
blue = (52 / 255, 168 / 255, 255 / 255)
gray = (203 / 255, 203 / 255, 203 / 255)
red = (255 / 255, 25 / 255, 65 / 255)
green = (40 / 255, 250 / 255, 40 / 255)
orange = (253 / 255, 145 / 255, 20 / 255)

In [ ]:
tar_layer = 1
plt.figure(figsize=(10, 4))
plt.xlim([0, max(time)])
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.xlabel(r'时间 $(s)$', fontname='SimSun', fontsize=10)
plt.ylabel(r'加速度 $(m/s^2)$', fontname='SimSun', fontsize=10)
plt.plot(time, a[tar_layer], color=blue, label="原结构")
plt.plot(time, a_c[tar_layer], color=red, label="TVMD减震结构")
font = {
    "family": "SimSun",
    'weight': 'normal',
    "size": 8
}
plt.legend(prop=font, loc='best')
pic_name = wave_name + "下" + str(tar_layer) + "层加速度时程响应"
plt.savefig(r'./pic/{}.png'.format(pic_name), dpi=600, bbox_inches='tight')
plt.show()
plt.close()

In [ ]:
plt.figure(figsize=(5, 8))
plt.xlim(0, max([max(drift), max(drift_c)]) * 1.3 * 100)
plt.ylim(0.8, 9.2)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.xlabel(r'层间位移角(%)', fontname='SimSun', fontsize=10)
plt.ylabel(r'楼层', fontname='SimSun', fontsize=10)
plt.plot(drift * 100, layer, color=blue, label="原结构", marker='o')
plt.plot(drift_c * 100, layer, color=red, label="TVMD减震结构", marker='o')
font = {
    "family": "SimSun",
    'weight': 'normal',
    "size": 8
}
plt.legend(prop=font, loc='best')
pic_name = wave_name + "下" + "层间位移角"
plt.savefig(r'./pic/{}.png'.format(pic_name), dpi=600, bbox_inches='tight')
plt.show()
plt.close()

In [ ]:
max([max(drift), max(drift_c)])

In [ ]:
# 计算比值
a_each_max = np.max(a, 1)
a_each_c_max = np.max(a_c, 1)
aa = np.max(a_each_c_max) / np.max(a_each_max)
bb = (np.sum(a_each_c_max) / np.size(a_each_c_max)) / (np.sum(a_each_max) / np.size(a_each_max))
cc = np.max(drift_c) / np.max(drift)
dd = (np.sum(drift_c) / np.size(drift_c)) / (np.sum(drift) / np.size(drift))

t = (aa + bb + cc + dd) / 4

In [ ]:
design_wave_names = ["Artificial_EQSignal1-Acc", "Artificial_EQSignal2-Acc", "RSN15_KERN_TAF021",
                         "RSN40_BORREGO_A-SON033", "RSN55_SFERN_BVP090", "RSN92_SFERN_WRP090", "RSN93_SFERN_WND143"]
design_wave_dt = [0.02, 0.02, 0.01, 0.005, 0.005, 0.005, 0.005]
design_wave_NPTS = [2000, 2000, 5435, 9045, 5330, 5955, 8000]
t_all = 0
design_wave_dir = "./waves_design_opensees/"
for i in range(len(design_wave_names)):
    wave_file = design_wave_dir + design_wave_names[i] + '.txt'
    a, drift = original_model(wave_file, design_wave_NPTS[i], design_wave_dt[i], 1)
    a_c, drift_c = TVMD_model([7.05796612e-01,1.90148951e-02,8.37558089e-01,1.96644904e-01,4.96909257e-01,6.04102519e-01,8.41429583e-01,3.15667579e-01,3.57480201e-01], [7.80241108e-01,7.46671414e-01,3.72422579e-01,4.22368395e-01,5.09204220e-01,5.35238292e-01,3.73389523e-01,6.11123544e-01,2.06264989e-01], [2.06750633e-03,3.97349592e-01,6.38372627e-01,5.43185626e-01,6.84256685e-01,3.68308048e-01,8.93078751e-04,8.06661210e-01,9.50197629e-01], wave_file, design_wave_NPTS[i], design_wave_dt[i], 1)
    # 计算比值
    a_each_max = np.max(a, 1)
    a_each_c_max = np.max(a_c, 1)
    aa = np.max(a_each_c_max) / np.max(a_each_max)
    bb = (np.sum(a_each_c_max) / np.size(a_each_c_max)) / (np.sum(a_each_max) / np.size(a_each_max))
    cc = np.max(drift_c) / np.max(drift)
    dd = (np.sum(drift_c) / np.size(drift_c)) / (np.sum(drift) / np.size(drift))
    t = (aa + bb + cc + dd) / 4
    print(t)
    t_all += t
t_all /= len(design_wave_names)

In [ ]:
t_all

In [ ]:
a = [1, 2]

In [ ]:
import random
x = random.uniform(0.01, 1)
x

In [ ]:
import os
files = os.listdir("waves_peer")

In [ ]:
files

In [ ]:
my_list = [None] * 3

In [ ]:
my_list

In [ ]:
my_list[0] = 2

In [ ]:
my_list

In [ ]:
np.random.random(9)

In [ ]:
np.random.uniform(0.01, 2, 9)
np.random.uniform(0.01, 0.5, 9)
np.random.uniform(0.01, 0.5, 9)

In [ ]:
import socket

hostname = socket.gethostname()

In [ ]:
hostname

In [ ]:
import datetime

datetime.datetime

In [ ]:
import datetime

now = datetime.datetime.now()
print("当前时间：", now)

In [ ]:
import psutil

pid = os.getpid()
process = psutil.Process(pid)
process_name = process.name()

print("当前进程名：", process_name)

In [ ]:
np.random.randint(0, 10000)

In [ ]:
not True

In [ ]:
from wxpy import *
bot = Bot()